In [13]:
import random
#import numpy as np
#import matplotlib.pyplot as plt
#import pandas as pd
#import streamlit as st

# 1 per unique device
class Device:
    def __init__(self):
        self.device_model = nextDevice()
        self.lifetime = int(databank.table.loc[databank.table[databank.table['Model Name'] == self.device_model].index, 'Life Time'])
        self.purchase_price = int(databank.table.loc[databank.table[databank.table['Model Name'] == self.device_model].index, 'Purchase Price'])
        self.credit_payback_duration = 36
        self.inventory_value = self.purchase_price

    def sell (self):
        finance.device_sales += self.purchase_price * 0.35
        counter.devices_total -= 1

    def reduceLifetime (self):
         self.lifetime = self.lifetime -1
    
    def depreciate (self):
        if self.lifetime == 0:
            self.inventory_value = 0
        else:
            self.inventory_value = self.inventory_value * (1-(1/self.lifetime))
        
# 1 per unique user
class User:
    def __init__(self):
        self.term = counter.nextTerm()
        self.membership_type = self.term
        self.left_forever = False

    def reduceTerm (self):
        self.term = self.term -1
   
    def pay (self):
        self.total_paid += 

# 1 per simulation
class Counter:
    def __init__(self):
        self.users_total = 0
        self.devices_total = 0
        self.terms = random.choices(databank.terms, weights = databank.term_weights, k = 100)
        self.devices = random.choices(databank.models, weights = databank.model_weights, k = 100)
    
    def nextDevice():
        return self.devices[self.users_total % 100]

    def nextTerm():
        return self.terms[self.users_total % 100]

counter = Counter()

# 1 per simulation
class Databank:
    def __init__(self):
        self.id_dictionary = {}
        self.inventory = {}
        self.table = pd.read_csv('Siradaki_Price_List.csv',sep=';')
        self.models = self.table['Model Name']
        self.model_weights = self.table['Rental Weights']
        self.terms = [1,3,6,12]
        self.term_weights = [5,5,10,80]
    
    def createMembership():
        user_id = "user_" + str(counter.users_total)
        device_id = "device_" + str(counter.devices_total)
        for k,v in inventory:
            if v == nextDevice() and globals()[k].lifetime > 0:
                id_dictionary[user_id,k]
                del inventory[v]
                buy_device = False
            else:
                buy_device = True
        if buy_device:
            createDevice()
            id_dictionary[user_id,device_id]
        createUser()

    def createUser():
        user_id = "user_" + str(counter.users_total)
        globals()[user_id] = User()
        counter.users_total += 1

    def createDevice():
        device_id = "user_" + str(counter.devices_total)
        globals()[device_id] = Device()
        counter.devices_total += 1

databank = Databank()

# sigorta + bakim eklenecek
class Finance:
    def __init__(self):
        self.device_sales = 0
        self.rent_income = 0
        self.credit_costs = 0
        self.gross = self.device_sales + self.rent_income - self.credit_costs - self.overheads

finance = Finance()
